# 2020-21 gameweek 1 predictions

## Set-up

In [2]:
import os

import pyarrow.parquet as pq

In [3]:
os.chdir('../..')

In [4]:
from src.models.DeepFantasyFootball.make_predictions import DeepFantasyFootball, load_live_data, SEASON_ORDER_DICT
from src.data.s3_utilities import s3_filesystem

Using TensorFlow backend.


## Make predictions

In [5]:
%%time
live_data = load_live_data()

2020-09-06 19:45:12,238 - Loading historical FFS data
2020-09-06 19:45:18,467 - Loaded historical FFS data of shape (92410, 66)
2020-09-06 19:45:18,468 - Loading latest FFS data
2020-09-06 19:45:18,819 - No data for 2020-2021 available, returning empty DataFrame for latest FFS data
2020-09-06 19:45:18,823 - Loaded latest FFS data of shape: Empty DataFrame
Columns: []
Index: []
2020-09-06 19:45:18,903 - Combined FFS data shape:       Team  Aerial Duels - Won - Percentage  Assists  Big Chances Created  \
0      ARS                             75.0        0                    0   
1      QPR                             40.0        0                    0   
2      ARS                              0.0        0                    0   
3      MUN                              0.0        0                    0   
4      CHE                              0.0        0                    1   
...    ...                              ...      ...                  ...   
92405  EVE                    

CPU times: user 1.94 s, sys: 799 ms, total: 2.73 s
Wall time: 13.6 s


In [6]:
live_data.head()

,name,team_name,season,Aerial Duels - Won - Percentage,Assists,Bad Touches,Big Chances Created,Caught Offside,Chances From Counter Attack,Clean Sheets,...,next_gameweek_win_odds,number_of_home_matches,number_of_top_6_last_season_opponent,position_DEF,position_FWD,position_GK,position_MID,top_6_last_season,total_points,season_order
0,aaron_connolly,Brighton and Hove Albion,2019-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.21,0.0,0,0,1,0,0,0,0.0,4
25,aaron_connolly,Brighton and Hove Albion,2019-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.34,1.0,0,0,1,0,0,0,0.0,4
50,aaron_connolly,Brighton and Hove Albion,2019-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.23,1.0,0,0,1,0,0,0,0.0,4
75,aaron_connolly,Brighton and Hove Albion,2019-20,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,1.14,0.0,1,0,1,0,0,0,1.0,4
100,aaron_connolly,Brighton and Hove Albion,2019-20,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,2.26,1.0,0,0,1,0,0,0,1.0,4


In [7]:
PREVIOUS_GW = 38
PREDICTION_SEASON = '2019-20'
PREDICTION_SEASON_ORDER = SEASON_ORDER_DICT[PREDICTION_SEASON]

In [8]:
lstm_pred = DeepFantasyFootball(
    previous_gw=PREVIOUS_GW,
    prediction_season_order=PREDICTION_SEASON_ORDER,
    previous_gw_was_double_gw=False  # Remove parameter
)

In [9]:
player_list, player_data_list = lstm_pred.prepare_data_for_lstm(full_data=live_data)

2020-09-06 19:47:40,139 - Number of players available for selection: 70443.0
2020-09-06 19:47:40,313 - Player data shape before: (70443, 66)
2020-09-06 19:47:40,427 - Player data shape after removing players with insufficient GW data: (70443, 66)
2020-09-06 19:47:40,983 - Player data shape after only keeping records needed for LSTM: (2575, 66)


In [10]:
unformatted_predictions = lstm_pred.make_player_predictions(
    player_data_list=player_data_list
)

2020-09-06 19:47:43,124 - LSTM input array shape: (515, 5, 62)


In [11]:
final_predictions = lstm_pred.format_predictions(
    player_list=player_list,
    final_predictions=unformatted_predictions,
    full_data=live_data,
    double_gw_teams=[]
)

In [12]:
final_predictions.head(20)

,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,name,position_DEF,position_FWD,position_GK,position_MID,team_name
362,5.075442,5.092628,5.211470,5.165286,5.033545,25.578373,mohamed_salah,0,0,0,1,Liverpool
417,4.765291,5.084347,4.875215,4.862520,4.754043,24.341417,raheem_sterling,0,0,0,1,Manchester City
190,4.645052,4.668022,4.687000,4.619446,4.524325,23.143845,harry_kane,0,1,0,0,Tottenham Hotspur
31,5.087066,4.402773,4.415803,4.244497,4.093180,22.243319,andrew_robertson,1,0,0,0,Liverpool
144,4.244996,4.470216,4.302823,4.281653,4.145956,21.445642,ederson_santana de moraes,0,0,1,0,Manchester City
199,4.494767,4.342222,4.313999,4.194793,4.088750,21.434532,hugo_lloris,0,0,1,0,Tottenham Hotspur
294,4.466159,4.439454,4.233912,4.149946,4.022650,21.312120,kevin_de bruyne,0,0,0,1,Manchester City
26,4.659870,4.245392,4.242934,4.112197,3.969017,21.229408,alisson_ramses becker,0,0,1,0,Liverpool
174,4.245816,4.417468,4.187945,4.192769,4.105789,21.149788,gabriel_fernando de jesus,0,1,0,0,Manchester City
231,4.330258,4.296165,4.205077,4.141763,3.968951,20.942215,jamie_vardy,0,1,0,0,Leicester City


In [13]:
final_predictions[final_predictions['sum'].isnull()]['team_name'].unique()

array(['Bournemouth', 'Watford', 'Norwich City'], dtype=object)

In [14]:
final_predictions[~final_predictions['sum'].isnull()].tail(20)

,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,name,position_DEF,position_FWD,position_GK,position_MID,team_name
278,0.151036,0.273338,0.337271,0.401805,0.449749,1.613199,josé_ignacio peleteiro ramallo,0,0,0,1,Aston Villa
420,0.138716,0.258155,0.321401,0.384036,0.428307,1.530617,ravel_morrison,0,0,0,1,Sheffield United
317,0.138716,0.258155,0.321401,0.384036,0.428307,1.530617,luke_freeman,0,0,0,1,Sheffield United
369,0.138716,0.258155,0.321401,0.384036,0.428307,1.530617,muhamed_besic,0,0,0,1,Sheffield United
213,0.138716,0.258155,0.321401,0.384036,0.428307,1.530617,jack_rodwell,0,0,0,1,Sheffield United
77,0.138716,0.258155,0.321401,0.384036,0.428307,1.530617,callum_robinson,0,0,0,1,Sheffield United
18,0.190241,0.235954,0.324293,0.364624,0.413711,1.528822,albian_ajeti,0,1,0,0,West Ham United
465,0.105339,0.272002,0.325358,0.393804,0.429192,1.525695,steven_alzate,0,0,0,1,Brighton and Hove Albion
69,0.165274,0.239388,0.319562,0.369488,0.422839,1.516550,borja_gonzález tomás,0,1,0,0,Aston Villa
501,0.165274,0.239388,0.319562,0.369488,0.422839,1.516550,wesley_moraes ferreira da silva,0,1,0,0,Aston Villa


In [15]:
final_predictions.groupby('position_MID').mean().loc[1]['sum']

7.591370105743408

In [16]:
final_predictions.groupby('position_FWD').mean().loc[1]['sum']

8.346404075622559

In [17]:
old_model_predictions = pq.read_table(
    's3://fpl-analysis-data/gw_predictions/season=2020-21/gw=1/95fe694d21c04484a2b2cab2d81419a0.parquet', 
    filesystem=s3_filesystem
).to_pandas()

In [18]:
old_model_predictions.head(20)

,GW_plus_1,GW_plus_2,GW_plus_3,GW_plus_4,GW_plus_5,sum,name,position_DEF,position_FWD,position_GK,position_MID,team_name,next_match_value,model
0,7.030556,6.761380,6.777621,6.667402,6.692840,33.929798,kevin_de bruyne,0,0,0,1,Manchester City,11.5,lstm_v4
1,6.319860,6.077927,6.090129,5.983685,6.004195,30.475794,raheem_sterling,0,0,0,1,Manchester City,11.5,lstm_v4
2,5.657178,5.443412,5.453287,5.351354,5.367841,27.273075,mohamed_salah,0,0,0,1,Liverpool,12.0,lstm_v4
3,4.935556,4.740576,4.740447,4.639941,4.648809,23.705330,gabriel fernando_de jesus,0,1,0,0,Manchester City,9.5,lstm_v4
4,4.922827,4.734571,4.737661,4.639350,4.649484,23.683895,sadio_mané,0,0,0,1,Liverpool,12.0,lstm_v4
5,4.798070,4.609890,4.609621,4.510372,4.518536,23.046490,bruno miguel_borges fernandes,0,0,0,1,Manchester United,10.5,lstm_v4
6,4.678135,4.494624,4.493752,4.395195,4.402573,22.464281,anthony_martial,0,1,0,0,Manchester United,9.0,lstm_v4
7,4.679602,4.493706,4.491284,4.391965,4.398650,22.455206,harry_kane,0,1,0,0,Tottenham Hotspur,10.5,lstm_v4
8,4.673827,4.489105,4.487793,4.388762,4.395951,22.435436,jamie_vardy,0,1,0,0,Leicester City,10.0,lstm_v4
9,4.516146,4.338603,4.335416,4.238196,4.243636,21.671997,pierre-emerick_aubameyang,0,0,0,1,Arsenal,12.0,lstm_v4
